In this question the behavior of algorithms developed to solve the Support Vector Machine problem are compared. The comparison is carried out on the breast cancer dataset of the sklearn package,

from sklearn.datasets import load_breast_cancer
The labels of the Breast Cancer dataset are of {0,1} which need to be converted into {−1,+1}. Scale each of the input variables to have the maximum absolute value equal to 1. For example, these steps can be implemented by
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

# load the data
X, y = load_breast_cancer(return_X_y=True)  ## X input, y output
print(X.shape, y.shape)

## to convert the {0,1} output into {-1,+1}
y = 2 * y -1

## X is the input matrix
mdata,ndim = X.shape
## normalization by L infinity norm
X/= np.outer(np.ones(mdata),np.max(np.abs(X),0))

## number of iteration
niter = 10

## penalty constant for the of the Stochastic Dual Coordinate Ascent algorithm
C = 1000
Three methods need to be applied. Those methods are the following ones:

The Stochastic Dual Coordinate Ascent for SVM algorithm which is described in the lecture slides. In this algorithm, use the linear kernel on the top of the input data. The penalty constant, C, is equal to 1000. Repeat 10 times the algorithm on the full training dataset.
 The method sklearn.svm.SVC from the sklearn package with polynomial kernel, denoted by poly in the sklearn. The degree of the polynomial is 5, and the penalty constant, C, here is equal to 1, the default.
The method sklearn.svm.SVC again from the sklearn package with Gaussian kernel, denoted by rbf in the sklearn. The gamma parameter of the rbf kernel is set to scale, and the penalty constant, C, here is also equal to 1, the default.
Process the data in the original order of the examples appearing in the data set. No training and test split is applied. The original full data is used in the training and in the test phases as well.

Compute the Area Under the Curve(AUC) score for the three versions of the SVM solution methods, thus for  the Stochastic Dual Coordinate Ascent for SVM, the sklearn.svm.SVC(kernel = "poly") and the sklearn.svm.SVC(kernel = "rbf"). Let the short names  of the methods be SDCA, SVCPoly, SVCRbf.

Question: What is the order of the performances of the methods measured by Area Under the Curve(AUC)?

Hint: You might apply the sklearn.metrics.roc_auc_score function to compute the AUC score. The sklearn also contains other alternative approaches.

SDCA < SVCRbf < SVCPoly

In [1]:
## ####################################################
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, f1_score, auc, roc_curve 

# load the data
X, y = load_breast_cancer(return_X_y=True)  ## X input, y output
print(X.shape, y.shape)

## to convert the {0,1} output into {-1,+1}
y = 2 * y -1

## X is the input matrix
mdata,ndim = X.shape
# ## normalization by L infinity norm
# X/= np.outer(np.ones(mdata),np.max(np.abs(X),0))

iscale = 1 ## =0, no scaling, =1 scaling the by the max absolute value
if iscale == 1: ## L_infty norm
    X /= np.outer(np.ones(mdata),np.max(np.abs(X),0))
elif iscale == 2: ## L2 norm
    xnorm = np.sqrt(np.sum(X**2,1))
    xnorm = xnorm + (xnorm==0)
    X /= np.outer(xnorm, np.ones(ndim))

## number of iteration 
niter = 10  

## penalty constant for the of the Stochastic Dual Coordinate Ascent algorithm
C = 1000

(569, 30) (569,)


In [4]:
## dual stochastic gradient algorithm
## initialize alpha
alpha = np.zeros(mdata)
## compute the linear kernel
K = np.dot(X,X.T)

for iter in range(niter):
    for i in range(mdata):
        ## process all sample examples sequentiallly
        ## sum on i!=j  => sum on all i-(i=j)
        alpha[i]=(1-y[i]*(np.sum(K[i]*y*alpha)-alpha[i]*y[i]*K[i,i]))/K[i,i]
        ## projection
        alpha[i]= min(C/mdata, max(0,alpha[i]))



In [7]:
## accuracy of the full data
Xtest =X
mtest = X.shape[0]
## Linear kernel
Kcross = np.dot(X, Xtest.T)
ysdca = np.sign(np.sum(np.outer(y*alpha, np.ones(mtest))*Kcross, 0))



In [10]:
# sdca_f1 = f1_score(y, ysdca)
sdca_auc = roc_auc_score(y,ysdca)
sdca_auc

0.9382366154008773

In [18]:
# 2. sklearn.svm.SVC poly,SVCPoly
clf2 = SVC(C=1, kernel='poly',degree=5)
clf2.fit(X,y)
y_pred = clf2.predict(X)

# roc_auc_score(y, clf.predict_proba(X)[:, 1])
roc_auc_score(y, clf2.predict(X))


0.9929245283018867

In [14]:
# 3. Gaussian kernel SVCRbf
clf3 = SVC(C=1, kernel='rbf',gamma='scale')
clf3.fit(X,y)
y_pred = clf3.predict(X)

# roc_auc_score(y, clf.predict_proba(X)[:, 1])
roc_auc_score(y, clf3.predict(X))

0.9773730246815707